# Making Use of Multiple GPUs

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [3]:
# In brief:
#device = torch.decive('cuda:0')
#model.to(device)
#my_tensor = my_tensor.to(device)
#model = nn.DataParallel(model)

In [4]:
INPUT_SIZE = 5
OUTPUT_SIZE = 2
BATCH = 30
DATA_SIZE = 100

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Dummy Data Set

In [6]:
class RandomDataSet(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len

In [7]:
rand_loader = DataLoader(dataset=RandomDataSet(INPUT_SIZE, DATA_SIZE), 
                         batch_size=BATCH, 
                         shuffle=True)

In [12]:
class Model(nn.Module):
    # define module...
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(INPUT_SIZE, OUTPUT_SIZE)
        
    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size: %s; output size: %s"
              % (input.size(), output.size()))
        return output

## Create Model and DataParallel

In [13]:
model = Model(INPUT_SIZE, OUTPUT_SIZE)
if torch.cuda.device_count() > 1:
    print('Using %d GPUs!' % torch.cuda.device_count())
    model = nn.DataParallel(model)

model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

## Run the Model

In [15]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size: %s; output size: %s" 
          % (input.size(), output.size()))

	In Model: input size: torch.Size([30, 5]); output size: torch.Size([30, 2])
Outside: input size: torch.Size([30, 5]); output size: torch.Size([30, 2])
	In Model: input size: torch.Size([30, 5]); output size: torch.Size([30, 2])
Outside: input size: torch.Size([30, 5]); output size: torch.Size([30, 2])
	In Model: input size: torch.Size([30, 5]); output size: torch.Size([30, 2])
Outside: input size: torch.Size([30, 5]); output size: torch.Size([30, 2])
	In Model: input size: torch.Size([10, 5]); output size: torch.Size([10, 2])
Outside: input size: torch.Size([10, 5]); output size: torch.Size([10, 2])


In [ ]:
# Above outputs will vary according to GPU presence and number.